In [1]:
import torch
import matplotlib.pyplot as plt
import math
import numpy as np
import IPython.display as ipd
from IPython.display import HTML
from jupyterthemes import jtplot
jtplot.style()
%config InlineBackend.figure_format = 'retina'

In [2]:
from adaptive_stft import optimize_stft

In [3]:
s = []
# s0 = torch.sin((2 * math.pi * torch.linspace(16, 4000, 8192) / 8192 * 650))
for i in [4, 8, 12]:
    x = torch.sin(torch.cumsum(2 * math.pi * torch.linspace(16, 8000 / (i + 1), int(8192 * 2 / (i + 1))) / (8192 * 2 / (i + 1)), 0))
    s.append(x)
rise = []
val = 0
for i in range(8192 + 1024):
    val += 1 / 8192
    if i > 4096:
        val += 4 / 8192 * math.pow((i - 4096) / 4096, 2)
    x = val * 500
    rise.append(math.sin(2 * math.pi * x))
s = torch.cat(s, dim=-1)
s = torch.cat((torch.from_numpy(np.array(rise)), s), dim=-1)

In [4]:
ipd.Audio(s.numpy(), rate=8192)

In [ ]:
import tqdm
if hasattr(tqdm.tqdm, '_instances'):
    tqdm.tqdm._instances.clear()
model, kur_hist, ani = optimize_stft(s.cuda(), lr=6e-5, num_windows=220, num_epochs=9000, window_shape='trapezoid', name_for_saving='sinesweep4_trape', make_animation=True)

In [ ]:
plt.plot(kur_hist)

In [ ]:
HTML(ani.to_html5_video())